In [ ]:
import datetime
import pandas as pd
import pyarrow as pa

In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
INPUT_DATA_FOLDER = "../data"
REPORTS_DATA_FILE = "ISW_vector.csv"

OUTPUT_FOLDER = "../data/all_data_preprocessed"
ISW_OUTPUT_DATA_FILE = "all_isw.csv"
ALARMS_OUTPUT_DATA_FILE = "all_alarms.csv"
WEATHER_EVENTS_OUTPUT_DATA_FILE = "all_weather_by_hour.csv"
MERGED_OUTPUT_DATA_FILE = "weather_regions_alarms_merged.csv"
ALL_MERGED_DATA_FILE = "all_merged"

MODEL_FOLDER = "model"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v1"
count_vectorizer_version = "v1"

In [ ]:
def isNaN(num):
    return num != num

## Reading data

In [ ]:
df_isw = pd.read_csv(f"{INPUT_DATA_FOLDER}/{REPORTS_DATA_FILE}", sep=",")

In [ ]:
df_isw.head(5)

## Preparing ISW reports

## Reading models

In [ ]:
df_isw.head(5)

In [ ]:
df_isw["report_date"] = pd.to_datetime(df_isw["date"])

In [ ]:
df_isw['date_tomorrow_datetime'] = df_isw['report_date'].apply(lambda x: x+datetime.timedelta(days=1))

In [ ]:
df_isw.drop("date", axis=1, inplace=True)

In [ ]:
df_isw.head(5)

In [ ]:
df_isw.to_csv(f"{OUTPUT_FOLDER}/{ISW_OUTPUT_DATA_FILE}", sep=";", index=False)

## Prepare events data

In [ ]:
EVENTS_DATA_FILE = "alarms.csv"

In [ ]:
df_events = pd.read_csv(f"{INPUT_DATA_FOLDER}/{EVENTS_DATA_FILE}", sep=";")

In [ ]:
df_events.head(5)

In [ ]:
df_events_v2 = df_events.drop(["id","region_id"],axis=1)

In [ ]:
df_events_v2.head(5)

In [ ]:
df_events_v2[isNaN(df_events_v2).any(axis=1)].head(5)

In [ ]:
df_events_v2["start"] = pd.to_datetime(df_events_v2["start"])
df_events_v2["end"] = pd.to_datetime(df_events_v2["end"])

In [ ]:
df_events_v2["start_hour"] = df_events_v2['start'].dt.floor('h')
df_events_v2["end_hour"] = df_events_v2['end'].dt.ceil('h')

In [ ]:
df_events_v2["day_date"] = df_events_v2["start"].dt.date

df_events_v2["start_hour_datetimeEpoch"] = df_events_v2['start_hour'].apply(lambda x: int(x.timestamp())  if not isNaN(x) else None)
df_events_v2["end_hour_datetimeEpoch"] = df_events_v2['end_hour'].apply(lambda x: int(x.timestamp())  if not isNaN(x) else None)

df_events_v2.head(10)

In [ ]:
df_events_v2[df_events_v2["all_region"]==1].shape

In [ ]:
df_events_v2[df_events_v2["all_region"]==1].head(5)

In [ ]:
df_events_v2[df_events_v2["all_region"]==0].shape

In [ ]:
df_events_v2[df_events_v2["all_region"]==0].head(5)

In [ ]:
df_events_v2.to_csv(f"{OUTPUT_FOLDER}/{ALARMS_OUTPUT_DATA_FILE}", sep=";", index=False)

## Prepare weather

In [ ]:
WEATHER_DATA_FILE = "weather_by_hour.csv"

In [ ]:
df_weather = pd.read_csv(f"{INPUT_DATA_FOLDER}/{WEATHER_DATA_FILE}", sep=",")
df_weather.head(5)

In [ ]:
df_weather["day_datetime"] = pd.to_datetime(df_weather["day_datetime"])

In [ ]:
df_weather.shape

In [ ]:
df_weather.head(10)

In [ ]:
# exclude
weather_exclude = [
"day_feelslikemax",
"day_feelslikemin",
"day_sunriseEpoch",
"day_sunsetEpoch",
"day_description",
"city_latitude",
"city_longitude",
"city_address",
"city_timezone",
"city_tzoffset",
"day_feelslike",
"day_precipprob",
"day_snow",
"day_snowdepth",
"day_windgust",
"day_windspeed",
"day_winddir",
"day_pressure",
"day_cloudcover",
"day_visibility",
"day_conditions",
"day_icon",
"day_source",
"day_preciptype",
"day_stations",
"hour_icon",
"hour_source",
"hour_stations",
"hour_feelslike"
]

In [ ]:
df_weather_v2 = df_weather.drop(weather_exclude, axis=1)

In [ ]:
df_weather_v2.head(5)

In [ ]:
df_weather_v2["city"] = df_weather_v2["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
df_weather_v2["city"] = df_weather_v2["city"].replace('Хмельницька область', "Хмельницький")

In [ ]:
df_weather_v2.head(5)

In [ ]:
df_weather_v2.shape

In [ ]:
df_weather_v2.to_csv(f"{OUTPUT_FOLDER}/{WEATHER_EVENTS_OUTPUT_DATA_FILE}", sep=";", index=False)

## merging data

In [ ]:
df_regions = pd.read_csv(f"../data/regions.csv")

In [ ]:
df_regions["region"] = df_regions["region"].apply(lambda x: x + " обл.")

In [ ]:
df_regions.head(5)

In [ ]:
df_weather_reg = pd.merge(df_weather_v2, df_regions, left_on="city",right_on="center_city_ua")

In [ ]:
df_weather_reg.head(10)

In [ ]:
df_weather_reg.shape

In [ ]:
df_weather_v2.shape

### Merging weather and events

In [ ]:
df_events_v2.dtypes

In [ ]:
df_events_v2.shape

In [ ]:
df_events_v2.head(10)

In [ ]:
df_regions["region"].unique()

In [ ]:
df_events_v2["region_city"].unique()

In [ ]:
# df_events_v2_sample = df_events_v2.sample(10)
# df_events_v2_sample.shape

events_dict = df_events_v2.to_dict('records')
events_by_hour = []

In [ ]:
events_dict[0]

In [ ]:
for event in events_dict:
    for d in pd.date_range(start=event["start_hour"], end=event["end_hour"], freq='1h'):
        et = event.copy()
        et["hour_level_event_time"] = d
        events_by_hour.append(et)

In [ ]:
df_events_v3 = pd.DataFrame.from_dict(events_by_hour)

In [ ]:
df_events_v3["hour_level_event_datetimeEpoch"] = df_events_v3["hour_level_event_time"].apply(lambda x: int(x.timestamp())  if not isNaN(x) else None)

In [ ]:
df_events_v3.shape

In [ ]:
df_events_v3.head(10)

In [ ]:
df_weather_reg.head(5)

In [ ]:
df_weather_reg.shape

In [ ]:
df_events_v3.head(10)

In [ ]:
df_events_v3["region_merge"] = df_events_v3["region_city"].apply(lambda x: "Київська обл." if x == "Київ" else x)

#count = df_events_v3.groupby("hour_level_event_time")['region_city'].nunique()
#df_events_v3['alarms_in_regions'] = df_events_v3["hour_level_event_time"].map(count)

#counts = df_events_v3.groupby(['region_city', 'day_date'])["region_city"].nunique()
#df_events_v3['events_on_day'] = df_events_v3.set_index(['region_city', 'day_date']).index.map(counts)

In [ ]:
df_events_v4 = df_events_v3.copy().add_prefix("event_")

In [ ]:
df_events_v4.head(10)

In [ ]:
df_weather_v4 = df_weather_reg.merge(df_events_v4, 
                                     how="left", 
                                     left_on=["region","hour_datetimeEpoch"],
                                     right_on=["event_region_merge","event_hour_level_event_datetimeEpoch"])

In [ ]:
df_weather_v4.drop(["event_region_merge"], axis=1, inplace=True)

In [ ]:
df_weather_v4.head(10)

In [ ]:
df_weather_v4.shape

In [ ]:
df_weather_v4.to_csv(f"{OUTPUT_FOLDER}/{MERGED_OUTPUT_DATA_FILE}", sep=";", index=False)

In [ ]:
df_weather_v4.head(10)

In [ ]:
df_weather_v4.shape

In [ ]:
df_isw.head(10)

In [ ]:
df_isw_v2 = df_isw.drop(["content", "stem_content"], axis=1)

In [ ]:
df_isw_v2.head(5)

In [ ]:
df_merged = df_weather_v4.merge(df_isw_v2, how="left", left_on="day_datetime", right_on="report_date")

In [ ]:
df_merged.head(10)

In [ ]:
df_merged=df_merged.drop(["city_resolvedAddress", "day_datetime", "city", "region", "event_day_date"], axis=1)
df_merged = df_merged.dropna(subset=df_isw_v2.columns.difference(["date"]))
df_merged = df_merged.dropna(subset=["event_all_region"])

In [ ]:
count = df_merged.groupby("event_hour_level_event_time")["event_region_city"].nunique()
df_merged['alarms_in_regions'] = df_merged["event_hour_level_event_time"].map(count)

In [ ]:
df_merged.shape

In [ ]:
df_merged.to_parquet(f"{OUTPUT_FOLDER}/{ALL_MERGED_DATA_FILE}.parquet", index=False, engine="pyarrow")

In [ ]:
# df_merged.to_csv(f"{OUTPUT_FOLDER}/{ALL_MERGED_DATA_FILE}.csv", index=False, sep=";")